# HW2

In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import OLSInfluence

from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

np.set_printoptions(precision=4, suppress=True)

## Q1

In [3]:
df = pd.read_csv('usina_with_outliers.csv')

print("Shape:", df.shape)
print("\nColumns:")
print(df.columns.tolist())

display(df.head())

print("\nSummary statistics:")
display(df.describe(include="all"))

print("\nMissing values per column:")
display(df.isna().sum())

Shape: (9568, 5)

Columns:
['AT', 'V', 'AP', 'RH', 'PE']


,AT,V,AP,RH,PE
0,14.96,41.76,1024.07,73.17,463.26
1,25.18,62.96,1020.04,59.08,444.37
2,5.11,39.40,1012.16,92.14,488.56
3,20.86,57.32,1010.24,76.64,446.48
4,10.82,37.50,1009.23,96.62,473.90



Summary statistics:


,AT,V,AP,RH,PE
count,9568.000000,9568.000000,9568.000000,9568.000000,9568.000000
mean,19.618518,54.250021,1013.288871,73.308978,454.407820
std,8.256412,13.993655,6.636609,16.094499,18.760047
min,-39.174839,-38.397358,959.607298,-53.091613,327.528030
25%,13.480000,41.670000,1009.077500,63.227500,439.730000
50%,20.320000,52.080000,1012.950000,74.955000,451.620000
75%,25.732500,66.540000,1017.320000,84.882500,468.530000
max,77.344839,155.117358,1064.772702,187.691613,590.091970



Missing values per column:


AT    0
V     0
AP    0
RH    0
PE    0
dtype: int64

In [ ]:
X = df.drop(columns=['PE'])
y = df['PE']

X_sm = sm.add_constant(X) 
model = sm.OLS(y, X_sm).fit()
infl = OLSInfluence(model)

cooks_dist = infl.cooks_distance[0]

outlier_indices = np.where(cooks_dist.to_numpy() > 4/len(y))[0]
df_clean = df.drop(outlier_indices)

df_clean.to_csv('usina.csv')
 


I chose the basic linear regression because I don't yet know much about the data here, and linear regression is the simplest model and requires less arbitrary choices.
I used Statsmodel because it has a built in for computing Cook's distance (at least with the straightforward linear regression).

## Q2

In [13]:
usina_outliers = pd.read_csv('usina_with_outliers.csv')
usina_clean = pd.read_csv('usina.csv')

def fit_eval_model(df, model):
    x = df.drop(columns=['PE'])
    y = df['PE']
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=344)

    model.fit(x_train, y_train)

    # Predict
    yhat_train = model.predict(x_train)
    yhat_test  = model.predict(x_test)

    # Metrics
    train_MSE = mean_squared_error(y_train, yhat_train)
    train_MAE = mean_absolute_error(y_train, yhat_train)
    train_R2 = r2_score(y_train, yhat_train)
    
    test_MSE = mean_squared_error(y_test, yhat_test)
    test_MAE = mean_absolute_error(y_test, yhat_test)
    test_R2 = r2_score(y_test, yhat_test)

    return [
        train_MSE, train_MAE, train_R2,
        test_MSE, test_MAE, test_R2
    ]

eval_outliers = []
eval_clean = []

eval_outliers.append(['Linear', 0] + fit_eval_model(usina_outliers, LinearRegression()))
eval_clean.append(['Linear', 0] + fit_eval_model(usina_clean, LinearRegression()))

lamda_vals = [0.01, 0.1, 1, 10, 100]

for lamda in lamda_vals:
    eval_outliers.append(['Ridge', lamda] + fit_eval_model(usina_outliers, Ridge(lamda)))
    eval_clean.append(['Ridge', lamda] + fit_eval_model(usina_clean, Ridge(lamda)))

for lamda in lamda_vals:
    eval_outliers.append(['Lasso', lamda] + fit_eval_model(usina_outliers, Lasso(lamda)))
    eval_clean.append(['Lasso', lamda] + fit_eval_model(usina_clean, Lasso(lamda)))

# Display table
eval_cols = [
    'Model', 'Lamda',
    'Train MSE', 'Train MAE', 'Train R^2',
    'Test MSE', 'Test MAE', 'Test R^2'
]

print("With outliers included:")
display(pd.DataFrame(eval_outliers, columns=eval_cols))

print("With outliers removed:")
display(pd.DataFrame(eval_clean, columns=eval_cols))

With outliers included:


,Model,Lamda,Train MSE,Train MAE,Train R^2,Test MSE,Test MAE,Test R^2
0,Linear,0.00,129.541861,5.228533,0.631435,110.838441,5.234288,0.685620
1,Ridge,0.01,129.541861,5.228533,0.631435,110.838441,5.234288,0.685620
2,Ridge,0.10,129.541861,5.228533,0.631435,110.838438,5.234288,0.685620
3,Ridge,1.00,129.541861,5.228540,0.631435,110.838404,5.234295,0.685620
4,Ridge,10.00,129.541861,5.228605,0.631435,110.838070,5.234359,0.685621
5,Ridge,100.00,129.541870,5.229254,0.631435,110.834739,5.235002,0.685630
6,Lasso,0.01,129.541864,5.228768,0.631435,110.840511,5.234492,0.685614
7,Lasso,0.10,129.542265,5.232298,0.631434,110.851420,5.237817,0.685583
8,Lasso,1.00,129.583484,5.268951,0.631317,110.996123,5.273298,0.685173
9,Lasso,10.00,132.521099,5.688432,0.622959,116.122291,5.689207,0.670633


With outliers removed:


,Model,Lamda,Train MSE,Train MAE,Train R^2,Test MSE,Test MAE,Test R^2
0,Linear,0.00,20.016985,3.599368,0.931355,20.048896,3.632437,0.930296
1,Ridge,0.01,20.016985,3.599368,0.931355,20.048896,3.632437,0.930296
2,Ridge,0.10,20.016985,3.599369,0.931355,20.048898,3.632438,0.930296
3,Ridge,1.00,20.016985,3.599371,0.931355,20.048925,3.632443,0.930296
4,Ridge,10.00,20.016986,3.599395,0.931355,20.049188,3.632500,0.930295
5,Ridge,100.00,20.017070,3.599641,0.931355,20.051889,3.633066,0.930286
6,Lasso,0.01,20.016989,3.599343,0.931355,20.048813,3.632404,0.930296
7,Lasso,0.10,20.017564,3.599703,0.931353,20.053876,3.633410,0.930279
8,Lasso,1.00,20.080522,3.607659,0.931137,20.161324,3.646186,0.929905
9,Lasso,10.00,25.813653,4.086092,0.911476,26.327242,4.153349,0.908468


The presence of outliers significantly increases both train and test error for all models evaluated.

The models appear to show better generalization with outliers removed. (However, some of this effect may be due to the no-outliers model being tested on more on cleaned data rather than the model itself behaving differently).

With outliers included, Ridge seems to slightly improve performance with lamda=100 (and have no effect with smaller values). Since the data is not scaled, it is possible that the optimal size of lamda is quite large, 100 could be around the peak of its effectiveness. Lasso regularization appears to worsen performance on both datasets, at least for large values of lamda.

## Q3

Using linear regression since it had the best performance on the cleaned dataset into the previous task.

Using statsmodels because it has simple functions available to calculate t- and p-values.

Using unscaled IV and DV for this task because the t-statistic used to test reliability already controls for the scale of the data. Since the model is plain linear regression, there is no regularization term that mismatched scaling can perturb.

In [22]:
x = usina_clean.drop(columns=['PE'])
y = usina_clean['PE']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=344)

ols = sm.OLS(y_train, sm.add_constant(x_train)).fit()

ols_table = pd.DataFrame({
    "beta_hat": ols.params,#.reshape(-1),
    "t_value": ols.tvalues,#.reshape(-1),
    "p_value": ols.pvalues#.reshape(-1)
}).sort_values("t_value", key=np.abs, ascending=False)

print(ols_table)

              beta_hat     t_value        p_value
AT           -1.938992 -106.571721   0.000000e+00
const       439.056906   37.648839  3.304458e-281
RH           -0.147628  -30.658008  4.139190e-193
V            -0.245466  -28.474463  2.531623e-168
AP            0.076581    6.767305   1.424743e-11
Unnamed: 0    0.000006    0.279362   7.799756e-01


The most reliably non-zero predictor is AT, whose t-value is over 100 in magnitude and whose p-value reports as zero. It is followed by RH, V, and AP, all of which strongly show statistical significance.

## Q4

Using linear regression since it had the best performance on the cleaned dataset on Part 2.

Using scikit because the relevant scaling and reporting features look  user-friendly.

Using scaled IV for this task because the slope of the output will be shallower on larger-scaled predictors, which is misleading. Scaled DV may improve readability but should not change results (interpretation) since there is only one set of Y-values.

In [47]:
x_scaler = StandardScaler()
x_train_sc = x_scaler.fit_transform(x_train)
x_test_sc = x_scaler.transform(x_train)

model = LinearRegression()
model.fit(x_train_sc, y_train)


cols = (['const'] + X.columns.tolist())
coeffs = (model.coef_)

print(pd.DataFrame([coeffs], columns=cols))

      const         AT        V        AP        RH
0  0.015379 -14.474018 -3.13677  0.456284 -2.181691
